#### Copyright 2019 Los autores de TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Clasificación sobre datos desequilibrados

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/imbalanced_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar cuaderno</a></td>
</table>

Este tutorial demuestra cómo clasificar un conjunto de datos altamente desequilibrado en el que la cantidad de ejemplos en una clase supera en gran medida a los ejemplos en otra. Trabajará con el conjunto de datos de [detección de fraudes de tarjetas de crédito](https://www.kaggle.com/mlg-ulb/creditcardfraud) alojado en Kaggle. El objetivo es detectar solo 492 transacciones fraudulentas de 284.807 transacciones en total. Utilizará [Keras](../../guide/keras/overview.ipynb) para definir el modelo y [las ponderaciones de clase](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model) para ayudar al modelo a aprender de los datos desequilibrados. .

Este tutorial contiene código completo para:

- Cargue un archivo CSV usando Pandas.
- Cree conjuntos de entrenamiento, validación y prueba.
- Defina y entrene un modelo usando Keras (incluido el establecimiento de pesos de clase).
- Evalúe el modelo utilizando varias métricas (incluida la precisión y la recuperación).
- Pruebe técnicas comunes para lidiar con datos desequilibrados como:
    - Ponderación de clase
    - Sobremuestreo


## Preparar

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Procesamiento y exploración de datos

### Descargue el conjunto de datos de fraude de tarjetas de crédito de Kaggle

Pandas es una biblioteca de Python con muchas utilidades útiles para cargar y trabajar con datos estructurados y se puede usar para descargar archivos CSV en un marco de datos.

Note: This dataset has been collected and analysed during a research collaboration of Worldline and the [Machine Learning Group](http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available [here](https://www.researchgate.net/project/Fraud-detection-5) and the page of the [DefeatFraud](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) project

In [ ]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

In [ ]:
raw_df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V26', 'V27', 'V28', 'Amount', 'Class']].describe()

### Examinar el desequilibrio de la etiqueta de clase

Veamos el desequilibrio del conjunto de datos:

In [ ]:
neg, pos = np.bincount(raw_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Esto muestra la pequeña fracción de muestras positivas.

### Limpiar, dividir y normalizar los datos

Los datos sin procesar tienen algunos problemas. Primero, las columnas `Time` y `Amount` son demasiado variables para usarlas directamente. Suelta la columna de `Time` (ya que no está claro qué significa) y toma el registro de la columna de `Amount` para reducir su rango.

In [ ]:
cleaned_df = raw_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps=0.001 # 0 => 0.1¢
cleaned_df['Log Ammount'] = np.log(cleaned_df.pop('Amount')+eps)

Divida el conjunto de datos en conjuntos de entrenamiento, validación y prueba. El conjunto de validación se utiliza durante el ajuste del modelo para evaluar la pérdida y cualquier métrica; sin embargo, el modelo no se ajusta a estos datos. El conjunto de prueba no se usa por completo durante la fase de entrenamiento y solo se usa al final para evaluar qué tan bien se generaliza el modelo a nuevos datos. Esto es especialmente importante con conjuntos de datos desequilibrados donde el [sobreajuste](https://developers.google.com/machine-learning/crash-course/generalization/peril-of-overfitting) es una preocupación importante por la falta de datos de entrenamiento.

In [ ]:
# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

Normalice las funciones de entrada con sklearn StandardScaler. Esto establecerá la media en 0 y la desviación estándar en 1.

Nota: `StandardScaler` solo se ajusta mediante `train_features` para asegurarse de que el modelo no se vea en los conjuntos de validación o prueba. 

In [ ]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)


Precaución: si desea implementar un modelo, es fundamental que conserve los cálculos de preprocesamiento. La forma más sencilla de implementarlos como capas y adjuntarlos a su modelo antes de exportarlos.


### Mira la distribución de datos

A continuación, compare las distribuciones de los ejemplos positivos y negativos de algunas características. Buenas preguntas que debe hacerse en este punto son:

- ¿Tienen sentido estas distribuciones?
    - Si. Ha normalizado la entrada y estos se concentran principalmente en el rango `+/- 2` .
- ¿Puedes ver la diferencia entre las distribuciones?
    - Sí, los ejemplos positivos contienen una tasa mucho mayor de valores extremos.

In [ ]:
pos_df = pd.DataFrame(train_features[ bool_train_labels], columns = train_df.columns)
neg_df = pd.DataFrame(train_features[~bool_train_labels], columns = train_df.columns)

sns.jointplot(pos_df['V5'], pos_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
plt.suptitle("Positive distribution")

sns.jointplot(neg_df['V5'], neg_df['V6'],
              kind='hex', xlim = (-5,5), ylim = (-5,5))
_ = plt.suptitle("Negative distribution")

## Definir el modelo y las métricas

Defina una función que cree una red neuronal simple con una capa oculta densamente conectada, una capa de [abandono](https://developers.google.com/machine-learning/glossary/#dropout_regularization) para reducir el sobreajuste y una capa sigmoidea de salida que devuelva la probabilidad de que una transacción sea fraudulenta: 

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

def make_model(metrics = METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(lr=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

### Comprender métricas útiles

Tenga en cuenta que hay algunas métricas definidas anteriormente que pueden ser calculadas por el modelo que serán útiles al evaluar el desempeño.

- **Los falsos** negativos y los **falsos** positivos son muestras que se clasificaron **incorrectamente**
- **Los verdaderos** negativos y **verdaderos** positivos son muestras que se clasificaron **correctamente**
- **La precisión** es el porcentaje de ejemplos clasificados correctamente

> $ \ frac {\ text {muestras verdaderas}} {\ text {muestras totales}} $

- **La precisión** es el porcentaje de positivos **previstos** que se clasificaron correctamente

> $ \ frac {\ text {verdaderos positivos}} {\ text {verdaderos positivos + falsos positivos}} $

- **El recuerdo** es el porcentaje de positivos **reales** que se clasificaron correctamente

> $ \ frac {\ text {verdaderos positivos}} {\ text {verdaderos positivos + falsos negativos}} $

- **AUC se** refiere al área bajo la curva de una curva característica de funcionamiento del receptor (ROC-AUC). Esta métrica es igual a la probabilidad de que un clasificador clasifique una muestra aleatoria positiva más alta que una muestra aleatoria negativa.

Nota: La precisión no es una métrica útil para esta tarea. Puede obtener una precisión superior al 99,8% en esta tarea si predice Falso todo el tiempo.

Lee mas:

- [Verdadero frente a falso y positivo frente a negativo](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)
- [Exactitud](https://developers.google.com/machine-learning/crash-course/classification/accuracy)
- [Precisión y recuperación](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall)
- [ROC-AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)

## Modelo de línea de base

### Construye el modelo

Ahora cree y entrene su modelo usando la función que se definió anteriormente. Tenga en cuenta que el modelo se ajusta con un tamaño de lote mayor que el predeterminado de 2048, esto es importante para garantizar que cada lote tenga una probabilidad decente de contener algunas muestras positivas. Si el tamaño del lote fuera demasiado pequeño, probablemente no tendrían transacciones fraudulentas de las que aprender.

Nota: este modelo no manejará bien el desequilibrio de clases. Lo mejorará más adelante en este tutorial.

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
model = make_model()
model.summary()

Prueba de ejecución del modelo:

In [ ]:
model.predict(train_features[:10])

### Opcional: establezca el sesgo inicial correcto.

Estas suposiciones iniciales no son buenas. Sabes que el conjunto de datos está desequilibrado. Establezca el sesgo de la capa de salida para reflejar eso (Ver: [Una receta para entrenar redes neuronales: "init well"](http://karpathy.github.io/2019/04/25/recipe/#2-set-up-the-end-to-end-trainingevaluation-skeleton--get-dumb-baselines) ). Esto puede ayudar con la convergencia inicial.

Con la inicialización de sesgo predeterminada, la pérdida debe ser de aproximadamente `math.log(2) = 0.69314` 

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

El sesgo correcto para establecer se puede derivar de:

$$ p_0 = pos / (pos + neg) = 1 / (1 + e ^ {- b_0}) $$ $$ b_0 = -log_e (1 / p_0 - 1) $$ $$ b_0 = log_e (pos / neg ) $$

In [ ]:
initial_bias = np.log([pos/neg])
initial_bias

Establezca eso como el sesgo inicial, y el modelo dará conjeturas iniciales mucho más razonables.

Debe estar cerca de: `pos/total = 0.0018`

In [ ]:
model = make_model(output_bias = initial_bias)
model.predict(train_features[:10])

Con esta inicialización, la pérdida inicial debería ser aproximadamente:

$$ - p_0log (p_0) - (1-p_0) log (1-p_0) = 0.01317 $$

In [ ]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Esta pérdida inicial es aproximadamente 50 veces menor que si hubiera sido con una inicialización ingenua.

De esta manera, el modelo no necesita pasar las primeras épocas simplemente aprendiendo que los ejemplos positivos son poco probables. Esto también facilita la lectura de gráficos de la pérdida durante el entrenamiento.

### Punto de control de los pesos iniciales

Para que las distintas ejecuciones de entrenamiento sean más comparables, mantenga los pesos de este modelo inicial en un archivo de punto de control y cárguelos en cada modelo antes del entrenamiento.

In [ ]:
initial_weights = os.path.join(tempfile.mkdtemp(),'initial_weights')
model.save_weights(initial_weights)

### Confirme que la corrección de sesgo ayuda

Antes de continuar, confirme rápidamente que la inicialización cuidadosa del sesgo realmente ayudó.

Entrene el modelo durante 20 épocas, con y sin esta inicialización cuidadosa, y compare las pérdidas: 

In [ ]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
model = make_model()
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [ ]:
def plot_loss(history, label, n):
  # Use a log scale to show the wide range of values.
  plt.semilogy(history.epoch,  history.history['loss'],
               color=colors[n], label='Train '+label)
  plt.semilogy(history.epoch,  history.history['val_loss'],
          color=colors[n], label='Val '+label,
          linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  
  plt.legend()

In [ ]:
plot_loss(zero_bias_history, "Zero Bias", 0)
plot_loss(careful_bias_history, "Careful Bias", 1)

La figura anterior lo deja claro: en términos de pérdida de validación, en este problema, esta inicialización cuidadosa da una clara ventaja. 

### Entrena el modelo

In [ ]:
model = make_model()
model.load_weights(initial_weights)
baseline_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels))

### Consultar historial de entrenamiento

En esta sección, producirá gráficos de la precisión y la pérdida de su modelo en el conjunto de entrenamiento y validación. Estos son útiles para verificar el sobreajuste, sobre el cual puede obtener más información en este [tutorial](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) .

Además, puede producir estos gráficos para cualquiera de las métricas que creó anteriormente. Los falsos negativos se incluyen como ejemplo.

In [ ]:
def plot_metrics(history):
  metrics =  ['loss', 'auc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()


In [ ]:
plot_metrics(baseline_history)

Nota: Que la curva de validación generalmente funciona mejor que la curva de entrenamiento. Esto se debe principalmente al hecho de que la capa de abandono no está activa al evaluar el modelo.

### Evaluar métricas

Puede utilizar una [matriz de confusión](https://developers.google.com/machine-learning/glossary/#confusion_matrix) para resumir las etiquetas reales frente a las predichas donde el eje X es la etiqueta predicha y el eje Y es la etiqueta real.

In [ ]:
train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

Evalúe su modelo en el conjunto de datos de prueba y muestre los resultados de las métricas que creó anteriormente.

In [ ]:
baseline_results = model.evaluate(test_features, test_labels,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_baseline)

Si el modelo hubiera predicho todo perfectamente, esta sería una [matriz diagonal](https://en.wikipedia.org/wiki/Diagonal_matrix) donde los valores fuera de la diagonal principal, que indican predicciones incorrectas, serían cero. En este caso, la matriz muestra que tiene relativamente pocos falsos positivos, lo que significa que hubo relativamente pocas transacciones legítimas que se marcaron incorrectamente. Sin embargo, es probable que desee tener incluso menos falsos negativos a pesar del costo de aumentar el número de falsos positivos. Esta compensación puede ser preferible porque los falsos negativos permitirían que se realicen transacciones fraudulentas, mientras que los falsos positivos pueden hacer que se envíe un correo electrónico a un cliente para pedirle que verifique la actividad de su tarjeta.

### Trazar la República de China

Ahora traza la [República de China](https://developers.google.com/machine-learning/glossary#ROC) . Este gráfico es útil porque muestra, de un vistazo, el rango de rendimiento que el modelo puede alcanzar simplemente ajustando el umbral de salida.

In [ ]:
def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right')

Parece que la precisión es relativamente alta, pero la recuperación y el área bajo la curva ROC (AUC) no son tan altas como le gustaría. Los clasificadores a menudo enfrentan desafíos cuando intentan maximizar tanto la precisión como la recuperación, lo cual es especialmente cierto cuando se trabaja con conjuntos de datos desequilibrados. Es importante considerar los costos de los diferentes tipos de errores en el contexto del problema que le preocupa. En este ejemplo, un falso negativo (se pierde una transacción fraudulenta) puede tener un costo financiero, mientras que un falso positivo (una transacción se marca incorrectamente como fraudulenta) puede disminuir la felicidad del usuario.

## Pesos de clase

### Calcular pesos de clase

El objetivo es identificar transacciones fraudulentas, pero no tiene muchas de esas muestras positivas con las que trabajar, por lo que le conviene que el clasificador tenga en cuenta los pocos ejemplos disponibles. Puede hacer esto pasando pesos de Keras para cada clase a través de un parámetro. Esto hará que el modelo "preste más atención" a ejemplos de una clase subrepresentada.

In [ ]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

### Entrena un modelo con pesos de clase

Ahora intente volver a entrenar y evaluar el modelo con ponderaciones de clase para ver cómo afecta eso a las predicciones.

Nota: El uso de `class_weights` cambia el rango de pérdida. Esto puede afectar la estabilidad del entrenamiento dependiendo del optimizador. Los optimizadores cuyo tamaño de paso depende de la magnitud del gradiente, como los `optimizers.SGD` , pueden fallar. El optimizador utilizado aquí, `optimizers.Adam` , no se ve afectado por el cambio de escala. También tenga en cuenta que debido a la ponderación, las pérdidas totales no son comparables entre los dos modelos.

In [ ]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_features, val_labels),
    # The class weights go here
    class_weight=class_weight) 

### Consultar historial de entrenamiento

In [ ]:
plot_metrics(weighted_history)

### Evaluar métricas

In [ ]:
train_predictions_weighted = weighted_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
weighted_results = weighted_model.evaluate(test_features, test_labels,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_weighted)

Aquí puede ver que con las ponderaciones de clase, la exactitud y la precisión son menores porque hay más falsos positivos, pero a la inversa, la recuperación y el AUC son mayores porque el modelo también encontró más verdaderos positivos. A pesar de tener una menor precisión, este modelo tiene una mayor recuperación (e identifica más transacciones fraudulentas). Por supuesto, ambos tipos de error tienen un costo (tampoco querrá molestar a los usuarios marcando demasiadas transacciones legítimas como fraudulentas). Considere cuidadosamente las compensaciones entre estos diferentes tipos de errores para su aplicación.

### Trazar la República de China

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')


plt.legend(loc='lower right')

## Sobremuestreo

### Sobremuestrear la clase minoritaria

Un enfoque relacionado sería volver a muestrear el conjunto de datos sobremuestreando la clase minoritaria.

In [ ]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

#### Usando NumPy

Puede equilibrar el conjunto de datos manualmente eligiendo el número correcto de índices aleatorios de los ejemplos positivos:

In [ ]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

In [ ]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

#### Usando `tf.data`

Si está utilizando `tf.data` la forma más fácil de producir ejemplos equilibrados es comenzar con un conjunto de datos `positive` y `negative` y fusionarlos. Consulte [la guía tf.data](../../guide/data.ipynb) para obtener más ejemplos.

In [ ]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

Cada conjunto de datos proporciona pares `(feature, label)` :

In [ ]:
for features, label in pos_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())

Combina los dos juntos usando `experimental.sample_from_datasets` :

In [ ]:
resampled_ds = tf.data.experimental.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

In [ ]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

Para usar este conjunto de datos, necesitará la cantidad de pasos por época.

La definición de "época" en este caso es menos clara. Supongamos que es la cantidad de lotes necesarios para ver cada ejemplo negativo una vez:

In [ ]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

### Entrene con los datos sobremuestreados

Ahora intente entrenar el modelo con el conjunto de datos remuestreados en lugar de usar ponderaciones de clase para ver cómo se comparan estos métodos.

Nota: Debido a que los datos se equilibraron replicando los ejemplos positivos, el tamaño total del conjunto de datos es mayor y cada época se ejecuta para más pasos de entrenamiento. 

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

resampled_history = resampled_model.fit(
    resampled_ds,
    epochs=EPOCHS,
    steps_per_epoch=resampled_steps_per_epoch,
    callbacks = [early_stopping],
    validation_data=val_ds)

Si el proceso de entrenamiento considerara el conjunto de datos completo en cada actualización de gradiente, este sobremuestreo sería básicamente idéntico a la ponderación de la clase.

Pero al entrenar el modelo por lotes, como lo hizo aquí, los datos sobremuestreados proporcionan una señal de gradiente más suave: en lugar de que cada ejemplo positivo se muestre en un lote con un gran peso, se muestran en muchos lotes diferentes cada vez con un pequeño peso.

Esta señal de gradiente más suave facilita el entrenamiento del modelo.

### Consultar historial de entrenamiento

Tenga en cuenta que las distribuciones de métricas serán diferentes aquí, porque los datos de entrenamiento tienen una distribución totalmente diferente de los datos de validación y prueba. 

In [ ]:
plot_metrics(resampled_history )

### Reentrenar


Debido a que el entrenamiento es más fácil con los datos balanceados, el procedimiento de entrenamiento anterior puede sobreajustarse rápidamente.

So break up the epochs to give the `callbacks.EarlyStopping` finer control over when to stop training.

In [ ]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

resampled_history = resampled_model.fit(
    resampled_ds,
    # These are not real epochs
    steps_per_epoch = 20,
    epochs=10*EPOCHS,
    callbacks = [early_stopping],
    validation_data=(val_ds))

### Vuelva a verificar el historial de entrenamiento

In [ ]:
plot_metrics(resampled_history)

### Evaluar métricas

In [ ]:
train_predictions_resampled = resampled_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(test_features, batch_size=BATCH_SIZE)

In [ ]:
resampled_results = resampled_model.evaluate(test_features, test_labels,
                                             batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_resampled)

### Trazar la República de China

In [ ]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')

plot_roc("Train Resampled", train_labels, train_predictions_resampled,  color=colors[2])
plot_roc("Test Resampled", test_labels, test_predictions_resampled,  color=colors[2], linestyle='--')
plt.legend(loc='lower right')

## Aplicando este tutorial a su problema

La clasificación de datos desequilibrada es una tarea intrínsecamente difícil, ya que hay muy pocas muestras de las que aprender. Siempre debe comenzar con los datos primero y hacer todo lo posible para recopilar tantas muestras como sea posible y pensar en profundidad sobre qué características pueden ser relevantes para que el modelo pueda aprovechar al máximo su clase minoritaria. En algún momento, su modelo puede tener dificultades para mejorar y producir los resultados que desea, por lo que es importante tener en cuenta el contexto de su problema y las compensaciones entre los diferentes tipos de errores.